In [1]:
# !pip3 install gpt-2-simple

In [2]:
from gpt_2_simple.src import model as gpt2_model, encoder
import tensorflow as tf
import numpy as np
import json

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
params = '117m-hparams.json'

In [4]:
hparams = gpt2_model.default_hparams()
with open(params) as f:
    hparams.override_from_dict(json.load(f))

with open('encoder.json', 'r') as f:
    en = json.load(f)
with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
    
bpe_merges = [
    tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]
]
enc_malay = encoder.Encoder(encoder=en, bpe_merges=bpe_merges)

In [5]:
batch_size = 2
maxlen = 1024
num_cpu_threads = 1

def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.compat.v1.io.parse_single_example(record, name_to_features)
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.compat.v1.int64:
            t = tf.compat.v1.to_int32(t)
        example[name] = t

    return example

name_to_features = {
    'input_ids': tf.compat.v1.io.FixedLenFeature([1024], tf.compat.v1.int64)
}
d = tf.compat.v1.data.TFRecordDataset(['gs://mesolitica-tpu-general/gpt2-testset/karangan.tfrecord'])
d = d.apply(
    tf.compat.v1.contrib.data.map_and_batch(
        lambda record: _decode_record(record, name_to_features),
        batch_size=batch_size,
        num_parallel_batches=num_cpu_threads,
        drop_remainder=True,
    )
)
d = d.make_one_shot_iterator().get_next()
d

Instructions for updating:
Use `tf.compat.v1.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.compat.v1.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.compat.v1.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.compat.v1.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `tf.compat.v1.cast` instead.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.compat.v1.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


{'input_ids': <tf.compat.v1.Tensor 'IteratorGetNext:0' shape=(2, 1024) dtype=int32>}

In [6]:
input_ids = d['input_ids']
output = gpt2_model.model(hparams=hparams, X=input_ids)
loss = tf.compat.v1.reduce_mean(
    input_tensor=tf.compat.v1.nn.sparse_softmax_cross_entropy_with_logits(
        labels=input_ids[:, 1:], logits=output['logits'][:, :-1]
    )
)
loss

<tf.compat.v1.Tensor 'Mean:0' shape=() dtype=float32>

In [7]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [8]:
var_list = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES)
saver = tf.compat.v1.train.Saver(var_list = var_list)
saver.restore(sess, 'gs://mesolitica-tpu-general/gpt2-117m/model.ckpt-435300')

INFO:tensorflow:Restoring parameters from gs://mesolitica-tpu-general/gpt2-117m/model.ckpt-435300


In [9]:
losses = []
while True:
    try:
        l = sess.run(loss)
        print(l)
        losses.append(l)
    except Exception as e:
        print(e)
        break

1.4431666
1.5809603
1.7055348
1.8630769
1.6984978
1.767755
1.8292981
1.7617803
1.8705689
1.731211
1.8388227
2.0299616
2.1151528
1.8211054
1.9403484
1.9824086
1.8927426
1.8779304
1.9722944
1.8426896
1.6888677
1.6522177
1.6237339
1.771958
1.9789941
2.311753
1.9610463
1.9511932
1.9196131
1.994197
1.8725495
1.6075294
1.8378532
2.0257928
1.9228168
1.7626024
1.7001889
1.6126157
1.678913
1.8125083
1.7683703
End of sequence
	 [[node IteratorGetNext (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'IteratorGetNext':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 664, in launch_in

In [12]:
import numpy as np

np.exp(np.mean(losses))

6.232461